# What i' ve learned from my first arbitrage experiences

A lot of posts have been coming up talking about how traders get rich from arbitrage opportunities in the crypto market. I was trying to arbitrage for months, but I could never get a profitable strategy, so I tell you the common problems I found there, and they might be helpful for those who are starting with this, so you don't have to travel the same painful path. Summarizing, what I found:
* Most of the arbitrage opportunities disspear after considering exchanges fees and commissions
* Most of arbitrage opportunities appear between exchanges with maintenance wallets, what makes deposits or withdrawls imposibles.
* Transaction times between exchanges cause arbitrage to become a lottery due to market volatility.

If you want to cotinue reading, you are going to find how to build an arbitrage monitor from scrath with python and some useful experiences.

## ¿What is an arbitrage?

[Arbitrage](https://en.wikipedia.org/wiki/Arbitrage) is the practice of taking advantage of a price difference between two or more markets. For example, if you find that stocks from some company are traded at lower price in some market that in others, then you can buy that stocks at that market, and sell them in other markets, at higher price, taking a profit from the price difference. Simple. Unfortunatly, too simply.

That simplicity gives many traders the opportunity to take advantage of the situation, and this way, increasing the demand of the stocks in markets where prices are lower and the same time increasing the offer in markets with higher prices. So,  unintentionally, they stabilize price inequality, decreasing the opportunities. And, more traders on a market, more *efficiciently* they work to stabilize the prices. In fact, finance models assume an [arbitrage-free](https://www.glynholton.com/notes/arbitrage-free-pricing/) condition, which implies that, in efficient markets, there are not arbitrage opportunities right there waiting for you.

But, let's be honest, cryto assets markets are realtively green. There are constantly appearing new exchanges with low trading volumnes. This market is far from being efficient.

## How to find an arbitrage opportunity in the crypto market?

There are so many tools to find an arbitrage opportunity.
The simplest, take a look at websites that monitor this opportunities (for example, [this](https://cryptocoincharts.info/arbitrage)).
The problem with this websites is that you can't control which exchanges and which assets you monitor. And, in practice, this is definitly something you need to do, because to take an advantage of an opportunity, you have to have your investment assets ni the right moment, in the rigth exchange.

So, let's build from scratch our own monitor that looks to the opportunities we really care. To do this, let's use the [ccxt](https://github.com/ccxt/ccxt) python library which allows us to connect to several exchanges (currently, 115) and trade cryptos in an standarized way (ccxt developers, you have really done a great job). We are going to use this library to request ask/bid prices for differents assets in differents exchanges, compare them and think about an arbitrage strategy.

Then let's get started. First at all, let's import the libraries:

In [1]:
import numpy as np
import ccxt

We are going to use Numpy later to make computations. Now, let's define some exchanges:

In [99]:
exchanges = ["allcoin", "binance", "bitbay", "bitcoincoid", "bitfinex", 
"bitflyer", "bitlish", "bitso", "bitstamp", "bittrex", "bleutrade", "btcmarkets", "btcturk", "bxinth", 
"cex", "cryptopia", "dsx", "exmo", "gatecoin", "gdax", "gemini", "hitbtc",
"huobipro", "kraken", "kucoin", "lakebtc", "livecoin", "mixcoins", "okex", "poloniex", "qryptos",
"quadrigacx", "southxchange", "therock", "tidex", "wex", "yobit", "zaif", "zb"]

exchanges = ["Allcoin", "Binance", "Bitfinex", "Bittrex", "Cex", "Cryptopia", "Exmo", "Gatecoin", "Hitbtc",
"Huobipro", "Kraken", "Kucoin", "Livecoin", "Okex", "Poloniex", "Qryptos", "Quadrigacx", "Southxchange", "Yobit"]

With *exchanges* defined, now we can intialize the correspondant clients to request data to them. You can define them like this:

In [127]:
allcoin = ccxt.allcoin()

Or better, you can try this and avoid the need to write a lot:

In [101]:
clients = []
for exchange in exchanges:
    exec("client=ccxt.%s()"%(exchange.lower()))
    clients.append(client)

Now, let's define some pairs of interest:

In [24]:
symbols = ["ADA/BTC", "BCH/BTC", "BTG/BTC", "BTS/BTC", "CLAIM/BTC", "DASH/BTC", "DOGE/BTC", "EDO/BTC", "EOS/BTC",
           "ETC/BTC","ETH/BTC", "FCT/BTC", "ICX/BTC", "IOTA/BTC", "LSK/BTC", "LTC/BTC", "MAID/BTC", "NEO/BTC",
           "OMG/BTC", "QTUM/BTC", "STR/BTC", "TRX/BTC","VEN/BTC", "XEM/BTC", "XLM/BTC", "XMR/BTC", "XRP/BTC", "ZEC/BTC"]

Then, let me define some helpful variables:

In [25]:
ask = np.zeros((len(symbols), len(clients)))
bid = np.zeros((len(symbols), len(clients)))

Finally, request the data to the clients. We are going to use the *fetch_order_book* function in each client, wich returns the full orderbook, but we are going to only take care about the *bid* and *ask* values at the top (this is a first approach, we could explore better options computing a mean value for our trading investment).

Assuming you have not modified the *exchanges* list, this lines could take a while. To speed up, go back and define less clients.

In [26]:
for row, symbol in enumerate(symbols):
    for col, client in enumerate(clients):
        
        try:
            book = client.fetch_order_book(symbol)
            ask[row, col] = book['asks'][0][0]
            bid[row, col] = book['bids'][0][0]
        except:
            pass

I've defined the request inside a *try* block because some symbols are not traded in all exchanges, and bad requests raise errors, which are not helpfuls to our task. Another source of error could be rate request limits, which we are just passing now. We can also implement a delay to decrease the request rate to each exchange. That delay should be adjusted depending on wich exchanges you are connecting, because each one has their own limits. To deep in this you can check each exchange documentation later. I've also defined the ask and bid savings inside the *try* block because some requests could return empty arrays, and we are trying to avoid deal with this now.

In an efficient approach, we don't want to wait that the hole loop finishes to look for an arbitrage opportunity, but for now, just leave it like this and continue exploring the data to understand what could be our best options.

Having defined the bid and ask values for each coin in each exchange, now let's define our strategy and compute the profits and losses. Our strategy to trade coin C between exchanges E1 and E2 implies to buy C at E1, transfer C coins to E2, and sell it there. We want to make all this steps as quicly as we can because we don't want that somebody takes advantage on us, so we are going to execute each trade *inmediatly*, where *inmediatly* means, buy at the lowest price somebody is willing to sell, (i.e., lowest ask value), and sell at the highest price somebody is willing to buy, (i.e., highest bid value). So, let's define the steps:

* At E1, set a buy market order for C
* Transfer the C coins to E2.
* At E2, set a sell market order for C
* Compute the BTC profit of your investment as:

    profit = (bid_E2 / ask_E1 - 1) x 100 %

In practice, you also should pay for fees in exchanges. So let me define a *fee* variable to compute more real profits. We can define an array or a dict of *fees* for each exchange, but for the sake of simplicity let me define  a general (conservative) percentual fee:

In [128]:
fee = 0.25

Let's compute this, and count how many profitable opportunities we found:

In [120]:
opportunities = []

for i, symbol in enumerate(symbols):
    for j1, exchange1 in enumerate(exchanges):
        for j2, exchange2 in enumerate(exchanges):
            
            profit = 0
            if j1 != j2 and ask[i, j1]>0:
                profit = ((bid[i, j2]*(1-fee/100)) / (ask[i, j1]*(1+fee/100)) - 1) * 100
                
                if profit>0:
                    opportunities.append([symbol, exchange1, ask[i, j1], exchange2, bid[i, j2], round(profit,2)])
                
print("Amount of profitable opportunities", len(opportunities))

Amount of profitable opportunities 108


We have found 108 opportunities! Promising, isn't it? Let's take a look at the best ones:

In [126]:
opportunities = sorted(opportunities, reverse=True, key=lambda x: x[5])
print(opportunities[:10])

[['QTUM/BTC', 'Bitfinex', 0.0017542, 'Qryptos', 0.00309319, 75.45],
 ['QTUM/BTC', 'Binance', 0.001756, 'Qryptos', 0.00309319, 75.27],
 ['QTUM/BTC', 'Huobipro', 0.001756, 'Qryptos', 0.00309319, 75.27],
 ['QTUM/BTC', 'Bittrex', 0.00176215, 'Qryptos', 0.00309319, 74.66],
 ['QTUM/BTC', 'Kucoin', 0.0017623, 'Qryptos', 0.00309319, 74.64],
 ['QTUM/BTC', 'Hitbtc', 0.00176638, 'Qryptos', 0.00309319, 74.24],
 ['QTUM/BTC', 'Livecoin', 0.00182869, 'Qryptos', 0.00309319, 68.3],
 ['BTG/BTC', 'Yobit', 0.00474999, 'Cex', 0.005787, 21.22],
 ['BTG/BTC', 'Yobit', 0.00474999, 'Binance', 0.005695, 19.3],
 ['BTG/BTC', 'Yobit', 0.00474999, 'Hitbtc', 0.005684, 19.07]]

At the time of writing this article the best option is to buy *QTUM/BTC* at *Bitfinex* and sell it at *Qryptos*. For each *QTUM/BTC* traded, we could get a 75.45% profit.

## The naive approach...

Now we have found some opportunities, we can start arbitraging. The first opportunities we found are all related to buy some *QTUM* in some exchanges and sell them in *Qryptos*. The return of that arbitrage is something like a 75%. 

75%!! I'm going to be trading all day. I'm going to became rich. So easy, today. Just for curiosity, let's search "Qryptos+qtum+deposit" on google to see if everything is going to be so easy when we will try to deposit my *QTUM* coins there. And I find this:

![fig](figs/qtum_deposit.png)

It seems like not everything is going so well. Let's look at the [third](https://qryptos.zendesk.com/hc/en-us/articles/360004206251-I-can-t-deposit-QTUM-to-my-QRYPTOS-account) link: FAQ *Qryptos* explainin why out wallet is not going to be compatible with our *QTUM* tokens bought out there.

So don't waste your time there, we have 101 more possibilities. Let's take the next one. We need to buy some *BTG* coins at *Yobit* and sell them on *Cex*. Wait a moment... The following opportunities also imply to sell the *BTG* coins bought at *Yobit*. Let me suspect that time.

I open my *Yobit* account and I try to withdrawl some *BTG* to see if everything works fine. When I click on the *deposit* button, I find this notification:

![fig](figs/yobit_btg.png)

Ok, we are not going anywhere around here. Let's look for another choices.

In my list I have another interesing opportunity: buy some *BCH* coins on *Binance* and sell them on *Yobit*. This time with less profit. So this time, it could be real. First at all, we need some available *BTC* at one of the first exchanges: *Binance* in order to buy there. If we luckly have it, we are starting with the right food. Otherwise, start praying for your *BTC* arriving before the opportunity dissappear.

Fortunately, I have some *BTC* at *Binance*. So, I buy 1 *BCH* there and then I open my *Yobiy* account to look 
for my for my *BCH* address in order to send it my new coins. When I click on the *deposit* button, I find this notification:

![fig](figs/bcc_yobit_w.png)

Guess what? Wallet is on maintenance. You can aware this by trying to withdrawl some coins, which gives you this notification:

![fig](figs/bcc_yobit_1.png)

So, I wait for 5 minutes, and I try again:

![fig](figs/bcc_yobit_2.png)

And nothing happends. Nothing in 5 minutes, 10 minutes or 30 minutes. Even if I had had a previous *BCH* address, deposits would have a very important temporary delay (beleave me, this is not my first time doing this). So, I have not another choice than keep my *BCH*. Anyway, *buy and hold* has been always a good trading strategy in crypto markets, at least until the last quarter of 2017.

So, let's try with another option. The last one, *BTG* from *Huobi* to *Cex*. What happends when I try to withdrawl *BTG* from *Huobi*? Another wallet on maintenance.

![fig](figs/btg_huobi.png)

## ...and the reallity show

I was trying to find a profitable arbitrage approach for months. And I'm not going to lie, I've never found it. There were some opportunities, but I could never got a consistent way to find them. I mean, just with a monitor, and not checking each wallet on each exchange by my own. Despite the fact that a lot of opportunities are filtered just at the beginning because the profits dissappears computing exchange fees and transference commissions, we have to deal with wallet maintenances.

And you are lucky if your exchange warns you about that maintenances. Some exchanges don't do it. For example, once I tried an arbitrage between some exchange and *Exmo*. I was waiting for 2 hours my *BTC* deposit on *Exmo*, and then I wrote them. Three hours later, they answered me:

![fig](figs/exmo_btc.png)

Finally, my *BTC* took 2 more days to be deposited. So, you never know if everything is going to be as planned when your transfer crypto assets.

And even when you find an arbitrage opportunity between echanges where everything works find, you have to deal with the transference time. Most exchanges don't allow you to trade cryptos until a good amount of confirmations. This time is almost always greater than tens of minutes. And if you are waiting that prices remain stable in that time, probably you have never been trading cryptos. When the arbitrage profit is something like a 2% (yes, don't expect to find a better opportunity without a maintanance wallet), and after exchange fees and transference commissions something like 1.5%, or even lower, expecting that prices don't move that amount in 20 minutes is out of one's mind. And crazy, if you know that there will be traders aware about that difference, starting algorithms that probably are going to take advantage of the situation faster, turning you away with empty hands.

## Some final recommnedations:

When you are analyzing arbitrage opportunities, always be aware of:

* Correct profit calculations considering:
    * maker/taker fees at the purchase exchange
    * transaction commissions
    * maket/taker fees
* Maintenance of wallets
* Time of transactions and estimated changing value in prices during this time
* Some exchanges don't allow you to withdrawl your funds unless you have done the full verification
* Do some research on google about that arbitrage, to see if there is something you are missing
* Always suspect if you find an arbitrage opportunity involving fiat to crypto pairs, like *BTC/USD* or *ETH/USD*. For exampe, *BTC/USD* today is being traded at \$7549 at *Bitfinex* and at $7721 at *Bithumb*. That sounds like another good opportunity, isn't it? I have bad news for you, once you transfer your *BTC* coins to *Bithumb*, you can only exchange them there with *KWC* (at an equivalent price of \$7721 *USD*). Only if you have a validated account, and if you have a bank account at South Korea you are going to be able to withdrawl your fiat money. And then think about and strategy to transfer your money from South Korea to your country.

Always, always suspect before giving away your money.
And, before I leave, I'm going to try once again to deposit my *BCH* coins on *Yobit*:

![fig](figs/bcc_yobit_8.png)

mmm not, not my day. 

In our next article, we are going to analyze a better way to approach arbitrage opportunities. I mean, this time for real.